In [1]:
from myhdl import *
from pygmyhdl import *

initialize()

@group
def adder_bit(a, b, c_in, sum_out, c_out):
    sum_a_b = Wire()
    xor_g(a, b, sum_a_b)
    xor_g(c_in, sum_a_b, sum_out)
    
    cry_a_b, cry_a_c, cry_b_c, cry_1 = Wire(), Wire(), Wire(), Wire()
    and_g(a, b, cry_a_b)
    and_g(a, c_in, cry_a_c)
    and_g(b, c_in, cry_b_c)
    or_g(cry_a_b, cry_a_c, cry_1)
    or_g(cry_b_c, cry_1, c_out)

@group
def adder(a, b, sum_out):
    num_bits = len(a)
    c = Bus(num_bits+1)
    for j in range(num_bits):
        adder_bit(a.o[j], b.o[j], c.o[j], sum_out.i[j], c.i[j+1])
    
a, b, sum_a_b = [Bus(4,name) for name in 'a b a+b'.split()]
adder_module = adder(a, b, sum_a_b)

def test():
    for a.next in [1, 2, 3, 4]:
        for b.next in [2, 4, 6, 8]:
            yield delay(1)
            
simulate(test())
#random_sim(16,a,b)
show_waveforms('a b a+b')

<class 'myhdl.StopSimulation'>: No more events


In [2]:
c, d, sum_c_d = [Bus(4,name) for name in 'c d c+d'.split()]
len(adder(c, d, sum_c_d))

6

In [3]:
a, b, c_in, sum_, c_out = [Wire() for _ in range(5)]
toVHDL(adder_bit, a, b, c_in, sum_, c_out)

In [4]:
sum_c_d_2 = Bus(4)
toVHDL(adder, c, d, sum_c_d_2)

In [5]:
@group
def unrolled_adder(a, b, sum_):
    num_bits = len(a)
    c = Bus(num_bits+1)
    adder_bit(a(0), b(0), c(0), sum_.i[0], c.i[1])
    adder_bit(a(1), b(1), c(1), sum_.i[1], c.i[2])
    adder_bit(a(2), b(2), c(2), sum_.i[2], c.i[3])
    adder_bit(a(3), b(3), c(3), sum_.i[3], c.i[4])
    
len(unrolled_adder(c, d, sum_c_d))

5

In [6]:
c, d, sum_c_d = [Bus(4,name) for name in 'c d c+d'.split()]
toVHDL(unrolled_adder, c, d, sum_c_d)